<a href="https://colab.research.google.com/github/Jiyang-Liu0/Data-structure-and-algrithm/blob/main/rag_skeleton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
!git clone https://github.com/hf-lin/ChatMusician.git

Cloning into 'ChatMusician'...
remote: Enumerating objects: 1748, done.
remote: Counting objects: 100% (1748/1748), done.
remote: Compressing objects: 100% (1255/1255), done.
remote: Total 1748 (delta 500), reused 1689 (delta 477), pack-reused 0 (from 0)
Receiving objects: 100% (1748/1748), 1.45 MiB | 4.21 MiB/s, done.
Resolving deltas: 100% (500/500), done.


In [3]:
cd ChatMusician

/content/ChatMusician


In [5]:
pwd

'/content/ChatMusician'

In [6]:
!pip install pyarrow

In [7]:
!chmod +x ./model/train/scripts/train.sh

In [8]:
!pip install deepspeed

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 17.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.9/422.9 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 3.7 MB/s eta 0:00:00
  Created wheel for deepspeed: filename=deepspeed-0.16.1-py3-none-any.whl size=1543916 sha256=5d183d00d0e5e6cffac5ef9295186881d581d5a49b03556d6e023282968a0f18
  Stored in directory: /root/.cache/pip/wheels/e8/a6/09/09e982334b832b202b1752644a6b314b922eb3897416846b7e
Successfully built deepspeed


In [10]:
import pandas as pd
import json

# 加载 JSON 文件
file_path = 'abc_training_dataset_10.json'
with open(file_path, 'r') as f:
    data = json.load(f)

# 转换为 DataFrame
df = pd.DataFrame(data)

# 添加 src 和 id 列
df['src'] = 'distilled'
df['id'] = range(len(df))

# 调整列顺序
df = df[['instruction', 'input', 'output', 'src', 'id']]

# 保存为 Parquet 文件
parquet_file_path_transformed = 'Parquet/abc_training_dataset_transformed.parquet'
df.to_parquet(parquet_file_path_transformed, engine='pyarrow')
print(f"转换后的 Parquet 文件已保存到: {parquet_file_path_transformed}")

转换后的 Parquet 文件已保存到: Parquet/abc_training_dataset_transformed.parquet


In [12]:
!python model/train/data_preprocess.py \
    -t m-a-p/ChatMusician-Base \
    -i uiash/abc_training_dataset_10 \
    -o datasets \
    --tokenize_fn sft

(…)training_dataset_transformed (1).parquet: 100% 6.67k/6.67k [00:00<00:00, 29.8MB/s]
Generating train split: 100% 10/10 [00:00<00:00, 1074.88 examples/s]
remove_column_names: ['instruction', 'input', 'output', 'src', 'id']
num_proc must be <= 10. Reducing num_proc to 10 for dataset of size 10.
Running tokenizer on dataset (num_proc=10): 100% 10/10 [00:00<00:00, 18.86 examples/s]
Saving the dataset (1/1 shards): 100% 10/10 [00:00<00:00, 1991.88 examples/s]
preprocess done


In [13]:
!./model/train/scripts/train.sh datasets_sft m-a-p/ChatMusician-Base

[2024-12-08 20:51:09,112] [INFO] [real_accelerator.py:219:get_accelerator] Setting ds_accelerator to cuda (auto detect)
2024-12-08 20:51:14.873438: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-08 20:51:14.892944: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-08 20:51:14.898969: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-08 20:51:14.913312: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other o

In [27]:
ls m-a-p/ChatMusician-Base


ls: cannot access 'm-a-p/ChatMusician-Base': No such file or directory


In [23]:
!python model/train/data_preprocess.py \
    -t m-a-p/ChatMusician-Base \
    -i m-a-p/MusicPile-sft \
    -o datasets_sft \
    --tokenize_fn sft

README.md: 100% 3.06k/3.06k [00:00<00:00, 20.5MB/s]
(…)-00000-of-00004-dd6f148bebde7637.parquet: 100% 247M/247M [00:05<00:00, 42.1MB/s]
(…)-00001-of-00004-796efb97edd48847.parquet: 100% 247M/247M [00:05<00:00, 42.6MB/s]
(…)-00002-of-00004-af1b1fa0af59ea77.parquet: 100% 246M/246M [00:05<00:00, 42.5MB/s]
(…)-00003-of-00004-473c192bc6c03561.parquet: 100% 246M/246M [00:05<00:00, 41.8MB/s]
Generating train split: 100% 1139473/1139473 [00:13<00:00, 87218.36 examples/s] 
Using only the first 1% of the dataset: 56973 samples
remove_column_names: ['instruction', 'input', 'output', 'src', 'id']
Running tokenizer on dataset (num_proc=64): 100% 56973/56973 [04:15<00:00, 222.73 examples/s] 
Saving the dataset (1/1 shards): 100% 56973/56973 [00:00<00:00, 75574.32 examples/s]
preprocess done


In [ ]:
!./model/train/scripts/train.sh datasets_sft m-a-p/ChatMusician-Base

In [ ]:
from transformers import LlamaTokenizer, LlamaForCausalLM

# tokenizer = LlamaTokenizer.from_pretrained("m-a-p/ChatMusician-Base", use_fast=False)
model = LlamaForCausalLM.from_pretrained("m-a-p/ChatMusician-Base", trust_remote_code=True)

print("Tokenizer and model loaded successfully!")


The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [17]:
!pip install --force-reinstall transformers==4.32.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 85.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.8/447.8 kB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 94.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 751.2/751.2 kB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.7/781.7 kB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 435.0/435.0 kB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 101.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━